<a href="https://colab.research.google.com/github/sivahariditty/CNN-Experiments/blob/main/SqueezNet_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [31]:
transform_train = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

transform_test = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

In [32]:
batch_size = 128
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


VGG - 16

In [33]:
from torchvision import models

In [34]:
sqnet = models.squeezenet1_0(pretrained=True)

In [35]:
for param in sqnet.parameters():
  param.requires_grad = False

In [36]:
sqnet.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))

In [37]:
for param in sqnet.parameters():
  if(param.requires_grad):
    print(param.shape)

torch.Size([10, 512, 1, 1])
torch.Size([10])


In [38]:
def evaluation(dataloader,model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [41]:
sqnet = sqnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt=optim.SGD(sqnet.parameters(),lr=0.5)

In [42]:
%%time
max_epochs = 16

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = sqnet(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()

        del inputs,labels,outputs
        torch.cuda.empty_cache()

        if i%100 == 0:
          print('Iteration %d loss : %0.2f'%(i,loss.item()))
        
    print('Epoch: %d/%d' % (epoch, max_epochs))

Iteration 0 loss : 2.50
Iteration 100 loss : 2.30


KeyboardInterrupt: ignored